In [1]:
import pandas as pd
df=pd.read_csv("D:\\PROJECT\\IT SERVICE\\data\\raw\\incident_event_log.csv")

This dataset uses DD/MM/YYYY.
if skip dayfirst=True, your times will be wrong

In [2]:
time_cols = [
    "opened_at",
    "sys_created_at",
    "sys_updated_at",
    "resolved_at",
    "closed_at"
]

for col in time_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)


In [3]:
agg_rules = {
    # identity
    "opened_at": "min",
    "resolved_at": "max",
    "closed_at": "max",

    # outcome
    "made_sla": "last",

    # operational counts
    "reassignment_count": "max",
    "reopen_count": "max",
    "sys_mod_count": "max",

    # categorical (stable across lifecycle)
    "category": "first",
    "subcategory": "first",
    "impact": "first",
    "urgency": "first",
    "priority": "first",
    "assignment_group": "first",

    # text / config
    "u_symptom": "first",
    "cmdb_ci": "first"
}


min(opened_at) → incident creation

max(resolved_at) → final resolution

last(made_sla) → final SLA outcome

max(reassignments) → total friction

first(category/priority) → classification decided early

In [4]:
incident_df = (
    df
    .groupby("number", as_index=False)
    .agg(agg_rules)
)



In [6]:
incident_df.shape

(24918, 16)

In [7]:
incident_df.number.nunique()

24918

In [ ]:
incident_df.shape
incident_df.head()


,number,opened_at,resolved_at,closed_at,made_sla,reassignment_count,reopen_count,sys_mod_count,category,subcategory,impact,urgency,priority,assignment_group,u_symptom,cmdb_ci
0,INC0000045,2016-02-29 01:16:00,2016-02-29 11:29:00,2016-03-05 12:00:00,True,0,0,4,Category 55,Subcategory 170,2 - Medium,2 - Medium,3 - Moderate,Group 56,Symptom 72,?
1,INC0000047,2016-02-29 04:40:00,2016-03-01 09:52:00,2016-03-06 10:00:00,True,1,0,8,Category 40,Subcategory 215,2 - Medium,2 - Medium,3 - Moderate,Group 70,Symptom 471,?
2,INC0000057,2016-02-29 06:10:00,2016-03-01 02:55:00,2016-03-06 03:00:00,True,0,0,6,Category 20,Subcategory 125,2 - Medium,2 - Medium,3 - Moderate,Group 70,Symptom 471,?
3,INC0000060,2016-02-29 06:38:00,2016-03-02 12:06:00,2016-03-07 13:00:00,True,0,0,3,Category 9,Subcategory 97,2 - Medium,2 - Medium,3 - Moderate,Group 25,Symptom 450,?
4,INC0000062,2016-02-29 06:58:00,2016-02-29 15:51:00,2016-03-05 16:00:00,False,1,0,7,Category 53,Subcategory 168,2 - Medium,2 - Medium,3 - Moderate,Group 70,Symptom 232,?


In [8]:
incident_df["resolution_time_hrs"] = (
    incident_df["resolved_at"] - incident_df["opened_at"]
).dt.total_seconds() / 3600


In [9]:
print("Total incidents:", len(incident_df))
print("\nSLA distribution:")
print(incident_df["made_sla"].value_counts())

print("\nResolution time stats (hours):")
print(incident_df["resolution_time_hrs"].describe())


Total incidents: 24918

SLA distribution:
made_sla
True     15803
False     9115
Name: count, dtype: int64

Resolution time stats (hours):
count    23362.000000
mean       178.171582
std        532.787772
min          0.000000
25%          0.416667
50%         22.100000
75%        148.479167
max       8070.166667
Name: resolution_time_hrs, dtype: float64


In [11]:
incident_df.to_csv(
    "D:\\PROJECT\\IT SERVICE\\data\\processed\\ucl_incident_master.csv",
    index=False
)

print(" Incident-level master dataset created.")


 Incident-level master dataset created.


In [ ]:
# # SLA breach rate by priority
# incident_df.groupby("priority")["made_sla"].value_counts(normalize=True)

# # SLA breach rate by impact
# incident_df.groupby("impact")["made_sla"].value_counts(normalize=True)

# # SLA breach rate by urgency
# incident_df.groupby("urgency")["made_sla"].value_counts(normalize=True)

# # SLA breach rate by category (top 10)
# incident_df["category"].value_counts().head(10).index
# incident_df[incident_df["category"].isin(
#     incident_df["category"].value_counts().head(10).index
# )].groupby("category")["made_sla"].value_counts(normalize=True)


category     made_sla
Category 20  True        0.761949
             False       0.238051
Category 23  True        0.524743
             False       0.475257
Category 26  True        0.710832
             False       0.289168
Category 32  True        0.760535
             False       0.239465
Category 37  True        0.671366
             False       0.328634
Category 42  True        0.801047
             False       0.198953
Category 46  True        0.503344
             False       0.496656
Category 53  True        0.629002
             False       0.370998
Category 57  True        0.612445
             False       0.387555
Category 9   True        0.600351
             False       0.399649
Name: proportion, dtype: float64